In [1]:
algorithm_names = [
    'filo',
    'filo+no_routemin',
    'ails'
]

instance_names = [
    'XLTEST-n1048-k139',
    'XLTEST-n2168-k625',
    'XLTEST-n3101-k685',
    'XLTEST-n4245-k164',
    'XLTEST-n5174-k170',
    'XLTEST-n5649-k365',
    'XLTEST-n6034-k1234',
    'XLTEST-n8575-k343',
]

In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib notebook

In [3]:

import matplotlib.font_manager as fm

# --- 可选：设置中文字体 ---
# 确保你的系统中有一个支持中文的字体
# 这里尝试查找几个常见的字体
# def set_chinese_font():
#     """
#     尝试设置一个支持中文的字体。
#     """
#     font_names = ['SimHei', 'Microsoft YaHei', 'Arial Unicode MS', 'Heiti TC']
#     for font_name in font_names:
#         try:
#             # 检查字体是否可用
#             fm.FontProperties(fname=fm.findfont(fm.FontProperties(family=font_name)))
#             plt.rcParams['font.sans-serif'] = [font_name]
#             plt.rcParams['axes.unicode_minus'] = False  # 解决负号显示问题
#             print(f"中文字体已设置为: {font_name}")
#             return
#         except:
#             continue
#     print("警告：未找到合适的中文字体，中文可能显示为方框。")
# 
# # 尝试设置中文字体
# set_chinese_font()
# -------------------------


def plot_search_curves(instance_name, algorithm_names, base_dir='.', output_dir='.'):
    """
    绘制指定实例在所有算法下的时间-Fitness搜索曲线。

    参数:
    instance_name (str): 要绘制的实例名称 (例如 'instance_A')。
    algorithm_names (list): 包含所有算法 (method) 名称的列表。
                            这些名称对应于 'base_dir' 下的子目录。
    base_dir (str, optional): 存放所有算法目录的基础路径。默认为当前目录 '.'。
    """
    
    plt.figure(figsize=(12, 7))
    
    print(f"--- 开始绘制实例: {instance_name} ---")
    
    found_data = False # 标记是否找到了任何数据

    # 1. 遍历所有算法 (method)
    for method in algorithm_names:
        method_dir = os.path.join(base_dir, method)
        
        # 检查算法目录是否存在
        if not os.path.isdir(method_dir):
            print(f"信息: 目录 {method_dir} 不存在, 跳过。")
            continue

        # 2. 查找文件：
        # 根据您的描述 "method/instance_name.csv"
        # 我们将查找完全匹配的文件名
        
        file_path = None
        for file in os.listdir(method_dir):
            if file.endswith('.csv') and instance_name in file:
                file_path = os.path.join(method_dir, file)

        
        
        # if not os.path.exists(file_path):
        if file_path is None:
            # 如果严格匹配的文件不存在，打印警告并跳过
            # (如果想实现“包含”，需要用 os.listdir 遍历，这里按您给的结构来)
            print(f"信息: 文件 {file_path} 未找到, 跳过。")
            continue

        # 3. 读取CSV文件
        try:
            # 使用 pandas 读取, sep='[;,]' 使用正则表达式匹配逗号或分号
            # header=None 表示文件没有标题行
            # names=['time', 'fitness'] 指定列名
            # engine='python' 是使用正则表达式分隔符所必需的
            data = pd.read_csv(
                file_path, 
                sep='[;,]', 
                header=None, 
                names=['time', 'fitness'],
                engine='python',
                on_bad_lines='skip' # 跳过格式错误的行
            )
            
            # 检查是否读到了空文件
            if data.empty:
                print(f"警告: 文件 {file_path} 为空, 跳过。")
                continue
                
            # 确保数据是数值类型
            data['time'] = pd.to_numeric(data['time'], errors='coerce')
            data['fitness'] = pd.to_numeric(data['fitness'], errors='coerce')
            
            # 丢弃转换失败的行 (如果存在)
            data.dropna(inplace=True)

            # 确保数据按时间排序
            data = data.sort_values(by='time')
            
            print(f"{method}: {len(data)} entries | best fitness: {data['fitness'].iloc[-1]}")
            found_data = True

            # 4. 绘制折线图
            # 使用 'label=method' 以便图例显示算法名称
            plt.plot(data['time'], data['fitness'], label=method, marker='o', markersize=2, linestyle='-')

        except pd.errors.EmptyDataError:
            print(f"警告: 文件 {file_path} 为空, 跳过。")
        except Exception as e:
            print(f"错误: 读取或处理 {file_path} 时出错: {e}")

    # 5. 美化和显示图表
    if not found_data:
        print(f"--- 实例 {instance_name} 未找到任何有效数据, 无法绘图。 ---")
        plt.close() # 关闭空白的图形窗口
        return

    plt.xlabel('Time')
    plt.ylabel('Fitness')
    plt.title(f'"{instance_name}" Time vs. Fitness')
    plt.legend(title='Method') # 添加图例
    plt.grid(True, linestyle='--', alpha=0.6) # 添加网格线
    plt.tight_layout() # 自动调整布局
    
    # 保存图像或显示图像
    output_filename = f"{output_dir}\plot_{instance_name}.png"
    plt.savefig(output_filename)
    print(f"--- 绘图完成: {instance_name}, 图像已保存至 {output_filename} ---")
    plt.show()


In [4]:
def plot_best_fitness(instance_name, algorithm_names, base_dir='.', output_dir='.'):
    """
    绘制指定实例在所有算法下的【最佳】Fitness收敛曲线（单调递减）。

    参数:
    instance_name (str): 要绘制的实例名称 (例如 'instance_A')。
    algorithm_names (list): 包含所有算法 (method) 名称的列表。
    base_dir (str, optional): 存放所有算法目录的基础路径。默认为 '.'。
    output_dir (str, optional): 图像保存目录。默认为 '.'。
    """
    
    plt.figure(figsize=(12, 7))
    
    print(f"--- 开始绘制【最佳Fitness】曲线: {instance_name} ---")
    
    found_data = False # 标记是否找到了任何数据

    # 1. 遍历所有算法 (method)
    for method in algorithm_names:
        method_dir = os.path.join(base_dir, method)
        
        if not os.path.isdir(method_dir):
            print(f"信息: 目录 {method_dir} 不存在, 跳过。")
            continue

        # 2. 查找文件
        file_path = None
        for file in os.listdir(method_dir):
            if file.endswith('.csv') and instance_name in file:
                file_path = os.path.join(method_dir, file)

        # if not os.path.exists(file_path):
        if file_path is None:
            print(f"信息: 文件 {file_path} 未找到, 跳过。")
            continue

        # 3. 读取CSV文件
        try:
            data = pd.read_csv(
                file_path, 
                sep='[;,]', 
                header=None, 
                names=['time', 'fitness'],
                engine='python',
                on_bad_lines='skip' 
            )
            
            if data.empty:
                print(f"警告: 文件 {file_path} 为空, 跳过。")
                continue
                
            data['time'] = pd.to_numeric(data['time'], errors='coerce')
            data['fitness'] = pd.to_numeric(data['fitness'], errors='coerce')
            data.dropna(inplace=True)
            
            # 确保数据按时间排序
            data = data.sort_values(by='time').reset_index(drop=True)
            
            if data.empty:
                print(f"警告: {file_path} 在清理后为空, 跳过。")
                continue

            # ****************************************************
            # ** 核心改动：计算单调递减的 "Best Fitness" **
            #
            # .cummin() 会计算到当前行为止的累积最小值。
            # 这确保了曲线只会下降或保持水平，绝不会上升。
            data['best_fitness'] = data['fitness'].cummin()
            # ****************************************************
            
            print(f"{method}: {len(data)} entries | best fitness: {data['fitness'].iloc[-1]}")
            found_data = True

            # 4. 绘制折线图 (使用 'best_fitness' 列)
            plt.plot(data['time'], data['best_fitness'], label=method, marker='o', markersize=2, linestyle='-')

        except pd.errors.EmptyDataError:
            print(f"警告: 文件 {file_path} 为空, 跳过。")
        except Exception as e:
            print(f"错误: 读取或处理 {file_path} 时出错: {e}")

    # 5. 美化和显示图表
    if not found_data:
        print(f"--- 实例 {instance_name} 未找到任何有效数据, 无法绘图。 ---")
        plt.close() # 关闭空白的图形窗口
        return

    plt.xlabel('Time')
    # Y轴标签更新
    plt.ylabel('Best Fitness') 
    # 标题更新
    plt.title(f'"{instance_name}" (Best Fitness vs. Time)') 
    plt.legend(title='Method') 
    plt.grid(True, linestyle='--', alpha=0.6) 
    plt.tight_layout() 
    
    # 确保输出目录存在
    os.makedirs(output_dir, exist_ok=True)
    
    # 保存图像或显示图像
    output_filename = os.path.join(output_dir, f"plot_best_fitness_{instance_name}.png")
    plt.savefig(output_filename)
    print(f"--- 绘图完成: {instance_name}, 图像已保存至 {output_filename} ---")
    plt.show()

In [5]:
plot_search_curves(instance_names[0], algorithm_names, base_dir='../results', output_dir='../figures/ablation+no_routemin')

<IPython.core.display.Javascript object>

--- 开始绘制实例: XLTEST-n1048-k139 ---
filo: 344 entries | best fitness: 124581
filo+no_routemin: 1145 entries | best fitness: 124754
ails: 265 entries | best fitness: 124134.0
--- 绘图完成: XLTEST-n1048-k139, 图像已保存至 ../figures/ablation+no_routemin\plot_XLTEST-n1048-k139.png ---


In [6]:
plot_best_fitness(instance_names[0], algorithm_names, base_dir='../results', output_dir='../figures/ablation+no_routemin')

<IPython.core.display.Javascript object>

--- 开始绘制【最佳Fitness】曲线: XLTEST-n1048-k139 ---
filo: 344 entries | best fitness: 124581
filo+no_routemin: 1145 entries | best fitness: 124754
ails: 265 entries | best fitness: 124134.0
--- 绘图完成: XLTEST-n1048-k139, 图像已保存至 ../figures/ablation+no_routemin\plot_best_fitness_XLTEST-n1048-k139.png ---


In [7]:
plot_best_fitness(instance_names[1], algorithm_names, base_dir='../results', output_dir='../figures/ablation+no_routemin')

<IPython.core.display.Javascript object>

--- 开始绘制【最佳Fitness】曲线: XLTEST-n2168-k625 ---
filo: 1363 entries | best fitness: 535929
信息: 文件 None 未找到, 跳过。
ails: 592 entries | best fitness: 529907.0
--- 绘图完成: XLTEST-n2168-k625, 图像已保存至 ../figures/ablation+no_routemin\plot_best_fitness_XLTEST-n2168-k625.png ---


In [8]:
plot_best_fitness(instance_names[2], algorithm_names, base_dir='../results', output_dir='../figures/ablation+no_routemin')

<IPython.core.display.Javascript object>

--- 开始绘制【最佳Fitness】曲线: XLTEST-n3101-k685 ---
filo: 1756 entries | best fitness: 565704
信息: 文件 None 未找到, 跳过。
ails: 749 entries | best fitness: 563123.0
--- 绘图完成: XLTEST-n3101-k685, 图像已保存至 ../figures/ablation+no_routemin\plot_best_fitness_XLTEST-n3101-k685.png ---


In [9]:
plot_best_fitness(instance_names[3], algorithm_names, base_dir='../results', output_dir='../figures/ablation+no_routemin')

<IPython.core.display.Javascript object>

--- 开始绘制【最佳Fitness】曲线: XLTEST-n4245-k164 ---
filo: 984 entries | best fitness: 192620
信息: 文件 None 未找到, 跳过。
ails: 854 entries | best fitness: 192413.0
--- 绘图完成: XLTEST-n4245-k164, 图像已保存至 ../figures/ablation+no_routemin\plot_best_fitness_XLTEST-n4245-k164.png ---


In [10]:
plot_best_fitness(instance_names[4], algorithm_names, base_dir='../results', output_dir='../figures/ablation+no_routemin')

<IPython.core.display.Javascript object>

--- 开始绘制【最佳Fitness】曲线: XLTEST-n5174-k170 ---
filo: 1301 entries | best fitness: 307984
信息: 文件 None 未找到, 跳过。
ails: 1244 entries | best fitness: 306328.0
--- 绘图完成: XLTEST-n5174-k170, 图像已保存至 ../figures/ablation+no_routemin\plot_best_fitness_XLTEST-n5174-k170.png ---


In [11]:
plot_best_fitness(instance_names[5], algorithm_names, base_dir='../results', output_dir='../figures/ablation+no_routemin')

<IPython.core.display.Javascript object>

--- 开始绘制【最佳Fitness】曲线: XLTEST-n5649-k365 ---
filo: 1772 entries | best fitness: 592365
信息: 文件 None 未找到, 跳过。
ails: 1779 entries | best fitness: 589837.0
--- 绘图完成: XLTEST-n5649-k365, 图像已保存至 ../figures/ablation+no_routemin\plot_best_fitness_XLTEST-n5649-k365.png ---


In [12]:
plot_best_fitness(instance_names[6], algorithm_names, base_dir='../results', output_dir='../figures/ablation+no_routemin')

<IPython.core.display.Javascript object>

--- 开始绘制【最佳Fitness】曲线: XLTEST-n6034-k1234 ---
filo: 3352 entries | best fitness: 999748
信息: 文件 None 未找到, 跳过。
ails: 1974 entries | best fitness: 991662.0
--- 绘图完成: XLTEST-n6034-k1234, 图像已保存至 ../figures/ablation+no_routemin\plot_best_fitness_XLTEST-n6034-k1234.png ---


In [13]:
plot_best_fitness(instance_names[7], algorithm_names, base_dir='../results', output_dir='../figures/ablation+no_routemin')

<IPython.core.display.Javascript object>

--- 开始绘制【最佳Fitness】曲线: XLTEST-n8575-k343 ---
信息: 文件 None 未找到, 跳过。
信息: 文件 None 未找到, 跳过。
ails: 1964 entries | best fitness: 231573.0
--- 绘图完成: XLTEST-n8575-k343, 图像已保存至 ../figures/ablation+no_routemin\plot_best_fitness_XLTEST-n8575-k343.png ---
